<a href="https://colab.research.google.com/github/phanimvsk/Capstone_Project/blob/Source-code/Model_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
nvidia-smi

Tue Nov 29 17:25:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-9t4elp6l/farm-haystack_2d0cc62fcef5435a83f11f5f1efe0b53
  Resolved https://github.com/deepset-ai/haystack.git to commit af06519fc4eb646ba77d97da261a7dd5ec3fc7f3
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-9t4elp6l/farm-haystack_2d0cc62fcef5435a83f11f5f1efe0b53


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [4]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [5]:
%%bash --bg
sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [6]:
import time
time.sleep(3)

In [7]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [8]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "/content/drive/MyDrive/input_data2"
#s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
#fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

docs = convert_files_to_docs(dir_path=doc_dir, clean_func=None, split_paragraphs=True)

print(docs)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.preprocessing:Converting /content/drive/MyDrive/input_data2/Raw_Data.txt


[<Document: {'content': 'After some deliberation we have chosen Python as our programming language. It is freely available and has been adopted as the programming language of\nchoice for many practitioners in data science and machine learning. It has many useful\npackages for data manipulation (often ported from R) and has been designed to be easy to\nprogram.Python is a high-level, general-purpose programming language.\nPython is dynamically-typed and garbage-collected. It supports multiple programming paradigms\nIt is objected oriented Language. Data Science is an interdisciplinary field that mines raw data, analyses it, and comes up with patterns that are used to \nextract valuable insights.Data Science combines statistics, maths, specialised programs, artificial intelligence, machine learning etc.To be able to truly understand data science and machine learning it\nis important to appreciate the underlying mathematics and statistics, as well as the resulting\nalgorithms.The purpose 

In [9]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [10]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

#reader = FARMReader(model_name_or_path="anas-awadalla/gpt2-span-head-few-shot-k-32-finetuned-squad-seed-4", use_gpu=True)
reader = FARMReader(model_name_or_path="phanimvsk/Edtech_lightweight", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'phanimvsk/Edtech_lightweight' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'phanimvsk/Edtech_lightweight' (Roberta model) from model hub.
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at phanimvsk/Edtech_lightweight and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [39]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)


In [12]:
# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    #query="who won the first noble prize in physics?", params={"Retriever": {"top_k": 2}}
     query= "explain sigmoidal logistic thing?", params={"Retriever": {"top_k": 3}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.11s/ Batches]


In [13]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'with a substantial “backstory” in the appendices.Having business knowledge of the problem you are solving is very important. Why because one needs to have proper knowledge, ask relevant questions & ask appropriate questions.', 'type': 'extractive', 'score': 0.007254982367157936, 'context': 'with a substantial “backstory” in the appendices.Having business knowledge of the problem you are solving is very important. Why because one needs to have proper knowledge, ask relevant questions & ask appropriate questions.', 'offsets_in_document': [{'start': 5331, 'end': 5555}], 'offsets_in_context': [{'start': 0, 'end': 224}], 'document_id': 'f299d5af3c9063f1a6e763ba99a821db', 'meta': {'name': 'Raw_Data.txt'}}>,
             <Answer {'answer': 'with a substantial “backstory” in the appendices. \nWhen we taught these courses, we noticed that students were eager to learn not only how\nto apply algorithms but also to understand how these algorithms actually work. How

In [14]:
from haystack.utils import print_answers

# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(prediction, details="minimum")


Query: explain sigmoidal logistic thing?
Answers:
[   {   'answer': 'with a substantial “backstory” in the appendices.Having '
                  'business knowledge of the problem you are solving is very '
                  'important. Why because one needs to have proper knowledge, '
                  'ask relevant questions & ask appropriate questions.',
        'context': 'with a substantial “backstory” in the appendices.Having '
                   'business knowledge of the problem you are solving is very '
                   'important. Why because one needs to have proper knowledge, '
                   'ask relevant questions & ask appropriate questions.'},
    {   'answer': 'with a substantial “backstory” in the appendices. \n'
                  'When we taught these courses, we noticed that students were '
                  'eager to learn not only how\n'
                  'to apply algorithms but also to understand how these '
                  'algorithms actually work. How